In [30]:
def discretize(grid_size=100):
    from itertools import product
    from pymor.core.config import config
    import numpy as np

    assert config.HAVE_DUNEXT
    assert config.HAVE_DUNEGDT

    # assemble system matrices - dune-gdt code
    ##########################################

    from dune.xt.common import init_mpi, DuneError
    try:
        init_mpi()
    except:
        pass

    from dune.xt.grid import HAVE_DUNE_ALUGRID
    assert HAVE_DUNE_ALUGRID
    from dune.xt.grid import (make_cube_grid__2d_simplex_aluconform as make_cube_grid,
                              make_boundary_info_on_leaf_layer as make_boundary_info)
    grid = make_cube_grid(lower_left=[0, 0], upper_right=[1, 1],
                          num_elements=[grid_size, grid_size],
                          num_refinements=1, overlap_size=[0, 0])
    boundary_info = make_boundary_info(grid, 'xt.grid.boundaryinfo.alldirichlet')

    from dune.xt.functions import make_checkerboard_function_1x1, make_constant_function_1x1

    background_values = [[1.]]*(9*9)
    channel_vert_values = [[0.]]*(9*9)
    channel_hor_values = [[0.]]*(9*9)
    force_values = [[0.]]*(9*9)
    force_values[10] = [1.]
    force_values[70] = [-1.]
    for ii in range(4, 9*9, 9):
        background_values[ii] = [0.]
        channel_vert_values[ii] = [1.]
    for ii in (36, 37, 38, 39, 41, 42, 43, 45):
        channel_hor_values[ii] = [1.]

    diffusion_functions = []
    diffusion_functions.append(make_checkerboard_function_1x1(
        grid_provider=grid, lower_left=[0, 0], upper_right=[1, 1],
        num_elements=[9, 9], values=background_values, name='background'))
    diffusion_functions.append(make_checkerboard_function_1x1(
        grid_provider=grid, lower_left=[0, 0], upper_right=[1, 1],
        num_elements=[9, 9], values=channel_vert_values, name='vertical channel'))
    diffusion_functions.append(make_checkerboard_function_1x1(
        grid_provider=grid, lower_left=[0, 0], upper_right=[1, 1],
        num_elements=[9, 9], values=channel_hor_values, name='horizontal channel'))

    one = make_constant_function_1x1(grid, 1.)
    force = make_checkerboard_function_1x1(
        grid_provider=grid, lower_left=[0, 0], upper_right=[1, 1],
        num_elements=[9, 9], values=force_values, name='force')
    
    #diffusion_functions[0].visualize(grid, 'leaf', -1, 'background', False)
    #diffusion_functions[1].visualize(grid, 'leaf', -1, 'channel', False)
    #force.visualize(grid, 'leaf', -1, 'force', False)

    from dune.gdt import HAVE_DUNE_FEM
    assert HAVE_DUNE_FEM
    from dune.gdt import (make_cg_leaf_part_to_1x1_fem_p1_space as make_space,
                          make_elliptic_matrix_operator_istl_row_major_sparse_matrix_double
                              as make_elliptic_matrix_operator,
                          make_dirichlet_constraints,
                          make_l2_volume_vector_functional_istl_dense_vector_double
                              as make_l2_volume_vector_functional,
                          make_system_assembler)

    space = make_space(grid)
    system_assembler = make_system_assembler(space)

    from dune.xt.la import HAVE_DUNE_ISTL
    assert HAVE_DUNE_ISTL
    from dune.xt.la import IstlDenseVectorDouble

    elliptic_operators = [make_elliptic_matrix_operator(diffusion_function, space)
                          for diffusion_function in diffusion_functions]
    for op in elliptic_operators:
        system_assembler.append(op)

    l2_force_functional = make_l2_volume_vector_functional(force, space)
    system_assembler.append(l2_force_functional)

    h1_product_operator = make_elliptic_matrix_operator(one, space)
    system_assembler.append(h1_product_operator)

    clear_dirichlet_rows = make_dirichlet_constraints(boundary_info, space.size(), False)
    set_dirichlet_rows = make_dirichlet_constraints(boundary_info, space.size(), True)
    system_assembler.append(clear_dirichlet_rows)
    system_assembler.append(set_dirichlet_rows)

    system_assembler.assemble()

    rhs_vector = l2_force_functional.vector()
    lhs_matrices = [op.matrix() for op in elliptic_operators]
    for mat in lhs_matrices:
        clear_dirichlet_rows.apply(mat)
    lhs_matrices.append(lhs_matrices[0].copy())
    lhs_matrices[-1].scal(0)
    set_dirichlet_rows.apply(lhs_matrices[-1])
    h1_0_matrix = h1_product_operator.matrix()
    set_dirichlet_rows.apply(h1_0_matrix)
    set_dirichlet_rows.apply(rhs_vector)

    # wrap everything as a pyMOR discretization
    ###########################################

    # dune-xt-la wrappers
    from pymor.bindings.dunext import DuneXTVectorSpace, DuneXTMatrixOperator
    # dune-gdt wrappers
    from pymor.bindings.dunegdt import DuneGDTVisualizer, DuneGDTpyMORVisualizerWrapper

    # define parameter functionals (same as in pymor.analyticalproblems.thermalblock)
    from pymor.operators.constructions import LincombOperator, VectorFunctional
    from pymor.parameters.functionals import ProjectionParameterFunctional
    parameter_functionals = [1.,
                             ProjectionParameterFunctional(component_name='channel',
                                                           component_shape=(2,),
                                                           coordinates=(0,)),
                             ProjectionParameterFunctional(component_name='channel',
                                                           component_shape=(2,),
                                                           coordinates=(1,))]

    # wrap operators
    ops = [DuneXTMatrixOperator(mat) for mat in lhs_matrices]
    op = LincombOperator(ops, parameter_functionals + [1.])
    rhs = VectorFunctional(DuneXTVectorSpace(IstlDenseVectorDouble, space.size()).make_array([rhs_vector]))
    h1_product = DuneXTMatrixOperator(h1_0_matrix, name='h1_0_semi')

    # build visualizer and discretization
    from pymor.discretizations.basic import StationaryDiscretization
    from pymor.parameters.spaces import CubicParameterSpace
    visualizer = DuneGDTVisualizer(space)
    parameter_space = CubicParameterSpace(op.parameter_type, 0.1, 1.)
    d = StationaryDiscretization(op, rhs, products={'h1_0_semi': h1_product},
                                 parameter_space=parameter_space,
                                 visualizer=visualizer)

    return d
